In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/semisupervised-paraphrase-pdf.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/semisupervised-news-paraphrase.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/semisupervised-wiki-paraphrase.json

In [2]:
import json

In [3]:
# files = ['semisupervised-paraphrase-pdf.json', 'semisupervised-news-paraphrase.json',
#         'semisupervised-wiki-paraphrase.json']

In [4]:
# from tqdm import tqdm

# results = []
# for file in files:
#     with open(file) as fopen:
#         data = json.load(fopen)

#     for i in tqdm(data):
#         if len(i) == 2:
#             if 'Abstrak :' in i[0] or 'Abstrak:' in i[0] or \
#             'Abstrak :' in i[1] or 'Abstrak:' in i[1]:
#                 continue
#             results.append(i)

100%|██████████| 850382/850382 [00:00<00:00, 871060.18it/s]


In [5]:
# import tensorflow as tf

# filename = f't5-data/semisupervised-paraphrase.tsv'
# with tf.io.gfile.GFile(filename, 'w') as outfile:
#     for i in range(len(results)):
#         outfile.write('%s\t%s\n' % (results[i][0], results[i][1]))

In [6]:
# with open('../summary/paraphrase-set.json', ) as fopen:
#     data = json.load(fopen)
# data.keys()

dict_keys(['before', 'test_before', 'after', 'test_after'])

In [7]:
# filename = f't5-data/paraphrase.tsv'
# with tf.io.gfile.GFile(filename, 'w') as outfile:
#     for i in range(len(data['before'])):
#         outfile.write('%s\t%s\n' % (data['before'][i], data['after'][i]))

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [9]:
def cnn_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            't5-data/paraphrase.tsv',
            't5-data/semisupervised-paraphrase.tsv',
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds


def cnn_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['parafrasa: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

t5.data.TaskRegistry.remove('cnn_dataset')
t5.data.TaskRegistry.add(
    'cnn_dataset',
    dataset_fn = cnn_dataset,
    splits = ['train'],
    text_preprocessor = [cnn_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [10]:
from tqdm import tqdm

@registry.register_problem
class Seq2Seq(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 32100
    
    @property
    def is_generate_per_split(self):
        return False
    
    @property
    def dataset_splits(self):
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 100,
        }]
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split
        
        nq_task = t5.data.TaskRegistry.get("cnn_dataset")
        ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 768, "targets": 768})
        
        for ex in tqdm(tfds.as_numpy(ds)):
            yield ex
                    
    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):
        
        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            sample["inputs"] = sample['inputs'].tolist()
            sample["targets"] = sample['targets'].tolist()
            yield sample

In [11]:
!rm -rf t2t-paraphrase/data

In [12]:
DATA_DIR = os.path.expanduser("t2t-paraphrase/data")
TMP_DIR = os.path.expanduser("t2t-paraphrase/tmp")
 
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [13]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

PROBLEM = 'seq2_seq'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

0it [00:00, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
99888it [01:00, 1575.60it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
199850it [02:02, 1828.37it/s]

...

INFO:tensorflow:Shuffling data...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.
